In [ ]:
using Distributed
ifelse(nprocs()<2, addprocs(9), nothing);

@everywhere using LinearAlgebra
@everywhere LinearAlgebra.BLAS.set_num_threads(1) # set number of threads for BLAS
# using LinearAlgebra
# LinearAlgebra.BLAS.set_num_threads(8) # set number of threads for BLAS

In [ ]:
using Plots
# using Revise
@everywhere using LatticeQM

# Tutorial: Twisted honeycomb lattices

## Model

### Bands

In [ ]:
klin = 9

n_moire_angle = 6
tz = 0.46

# n_moire_angle = 11
# tz = 0.46

# n_moire_angle = 10
# tz = 0.51

# n_moire_angle = 13
# tz = 0.39

N_el_per_moire = -6.0
nothing

In [ ]:
@time "lattice" lat = Geometries.honeycomb_twisted(n_moire_angle)
# Geometries.smoothdisplaceZ!(lat, 0.045)
plot_supercells = Structure.Lattices.getneighborcells(lat, 1; halfspace=false, innerpoints=true, excludeorigin=false)
p = plot(lat, 3; supercell=plot_supercells, sort="layer", markersize=1.5)
display(p)

@time "valley_op" valley = Operators.valley(lat; spinhalf=false)
@time "graphene_op" H = Operators.graphene(lat; tz=tz, format=:sparse, mode=:nospin) # specifying format is important here!

# filling = 0.5 + 6.0 / hopdim(hops) # for spinful
filling = 0.5 + N_el_per_moire / 2 / hopdim(H) # for spinful
Operators.setfilling!(H, filling; nk=klin^2, multimode=:distributed) # less than 20s on Apple M1 single-core

ks = kpath(lat, ["μ2", "γ", "κ", "κ'", "γ", "μ"]; num_points=100)
bands = getbands(H, ks, valley; format=:sparse, multimode=:distributed, num_bands=25)

In [ ]:
p1 = plot(bands, 1; size=(300, 250), markercolor=:diverging_bkr_55_10_c35_n256, colorbar=true)
p2 = plot(bands, 1; ylim=(-0.14, 0.08), size=(300, 250), markercolor=:diverging_bkr_55_10_c35_n256, colorbar=true)

plot(p1, p2; size=(600, 200))

In [ ]:
kgrid = Structure.regulargrid(; nk=klin^2)
ωs = collect(range(-0.002, length=100, stop=0.002))
ldos = Spectrum.ldos(H, kgrid, ωs; Γ=0.0005, format=:sparse, num_bands=25);
plot(lat, ldos; supercell=plot_supercells, clims=(0, maximum(ldos)), markercolor=:inferno, colorbar=true, sort="layer", markersize=1.5)

# Mean-field iteration

In [ ]:
@time "valley_op" valley = Operators.valley(lat; spinhalf=true)
@time "graphene_op" H = Operators.graphene(lat; tz=tz, format=:sparse, mode=:spinhalf) # specifying format is important here!

# filling = 0.5 + 6.0 / hopdim(hops) # for spinful 
filling = 0.5 + N_el_per_moire / hopdim(H) # for spinful 
μ = Operators.setfilling!(H, filling; nk=klin^2) # less than 20s on Apple M1 single-core 

ks = kpath(lat, ["μ2", "γ", "κ", "κ'", "γ", "μ"]; num_points=75) 
bands = getbands(H, ks, valley; format=:sparse, num_bands=36) 
nothing 

In [ ]:
v = Operators.gethubbard(lat; mode=:σx, a=0.5, U=2.5) 
ρ_init = Meanfield.initialguess(v, :ferro; lat=lat) 

In [ ]:
ρ_sol, ϵ_GS, HMF, converged, residue = Meanfield.solvehartreefock( # run the calculation
    H, v, ρ_init, filling; klin=klin, iterations=15, tol=1e-3,# p_norm=Inf,
    T=0.001, β=0.85, show_trace=true, verbose=false, multimode=:distributed
)

In [ ]:
hmf = deepcopy(Meanfield.hMF(HMF))
Operators.addchemicalpotential!(hmf, -HMF.μ)
sx, sy, sz = Operators.getoperator(lat, ["sx", "sy", "sz"])

In [ ]:
bands = getbands(hmf, ks, [sz, valley]; format=:sparse, num_bands=36, multimode=:distributed)

plot(bands; size=(300, 250), markercolor=:diverging_bkr_55_10_c35_n256, colorbar=true)

In [ ]:
M = Operators.localobservables(ρ_sol, lat)
M = real(M)

In [ ]:
clim = max(abs.(M[1, :])...)
plot(lat, M[1, :]; supercell=plot_supercells, clims=(-clim, clim), markercolor=:RdBu, colorbar=true, sort="layer", markersize=1.5)

In [ ]:
LatticeQM.Lattices.countorbitals(lat)

In [ ]:
kgrid = Structure.regulargrid(; nk=9^2)
ωs = collect(range(-0.003, length=100, stop=0.003))
ldos = Spectrum.ldos(hmf, kgrid, ωs; Γ=0.0005, format=:sparse, num_bands=25);

ldos0 = ldos[1:2:hopdim(H)] + ldos[2:2:hopdim(H)]
ldosM = ldos[1:2:hopdim(H)] - ldos[2:2:hopdim(H)]
# 
plot(lat, ldos0; supercell=plot_supercells, clims=(0, maximum(ldos)), markercolor=:inferno, colorbar=true, sort="layer", markersize=1.5)

In [ ]:
ldosM = ldos[1:2:hopdim(H)] - ldos[2:2:hopdim(H)]
cmax = max(abs.(ldosM)...)
plot(lat, ldosM; supercell=plot_supercells, clims=(-cmax, cmax), markercolor=:RdBu, colorbar=true, sort="layer", markersize=1.5)

# Mean-field iteration (SC)

In [ ]:
@time "graphene_op" H = Operators.graphene(lat; tz=tz, format=:sparse, mode=:spinhalf) # specifying format is important here!

# filling = 0.5 + 6.0 / hopdim(hops) # for spinful 
filling = 0.5 + N_el_per_moire / hopdim(H) # for spinful 
μ = Operators.setfilling!(H, filling; nk=klin^2) # less than 20s on Apple M1 single-core 

hBDG = BdGOperator(H)
eOP = Superconductivity.electron(hBDG)

ks = kpath(lat, ["μ2", "γ", "κ", "κ'", "γ", "μ"]; num_points=75) 
bands = getbands(hBDG, ks, eOP.h; format=:sparse, num_bands=36)
nothing 

In [ ]:
plot(bands; size=(300, 250), markercolor=:diverging_bkr_55_10_c35_n256, colorbar=true)

In [ ]:
# v = Operators.gethubbard(lat; mode=:σx, a=0.5, U=2.0)
v = Operators.gethubbard(lat; mode=:σx, a=0.5, U=2.0)

ρ0_init = initialguess(v, :random, :nonlocal)
Δ0_init = initialguess(v, :random, :nonlocal)
ρ_init = BdGOperator(ρ0_init, Δ0_init)

ρ_sol, ϵ_GS, HMF, converged, error = Meanfield.solvehartreefock( # run the calculation
    hBDG, v, ρ_init, filling; klin=klin, iterations=25, tol=1e-5,# p_norm=Inf,
    T=0.001, β=0.75, show_trace=true, multimode=:distributed)
nothing

In [ ]:
hmf = deepcopy(Meanfield.hMF(HMF))
Operators.addchemicalpotential!(hmf, -HMF.μ)
sx, sy, sz = Operators.getoperator(lat, ["sx", "sy", "sz"])

In [ ]:
bands = getbands(hmf, ks, [sz, valley]; format=:sparse, num_bands=36, multimode=:distributed)

plot(bands; size=(300, 250), markercolor=:diverging_bkr_55_10_c35_n256, colorbar=true)